In [1]:
import sys

sys.path.append("/home/hacene/Documents/information_extraction_from_id_cards/src")

In [2]:
from os import listdir
from typing import List

import cv2
import imutils
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

from data.utils_old import get_characters_from_image

2022-01-05 15:12:46.422840: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/hacene/.cache/pypoetry/virtualenvs/ocr-id-cards-tiqq-sx6-py3.8/lib/python3.8/site-packages/cv2/../../lib64:
2022-01-05 15:12:46.422860: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
CLASSES = [
    "0",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9",
    "A",
    "B",
    "C",
    "D",
    "E",
    "F",
    "G",
    "H",
    "I",
    "J",
    "K",
    "L",
    "M",
    "N",
    "O",
    "P",
    "Q",
    "R",
    "S",
    "T",
    "U",
    "V",
    "W",
    "X",
    "Y",
    "Z",
    "<",
]

In [6]:
def return_class(predictions, CLASSES):
    mask = np.ones_like(predictions)
    classes = np.array(CLASSES)
    indices_m_and_n = np.where(((classes != "M") & (classes != "F")))[0]
    mask[0:15, 10:] = -1
    mask[16, indices_m_and_n] = -1
    mask[17:24, 10:] = -1
    mask[24:, 0:10] = -1
    classes = np.array(CLASSES)
    constrainted_prediction = np.multiply(predictions, mask)
    ind = np.argmax(constrainted_prediction, axis=1)
    return classes[ind]

In [7]:
def get_used_characters(characters):
    line_01 = [characters[0][i] for i in range(5, 14)]
    line_02 = [characters[1][i] for i in range(0, 15)]
    line_03 = characters[2]
    images = line_01 + line_02 + line_03
    return images

In [8]:
model = tf.keras.models.load_model("../models/model.h5")

2022-01-05 00:28:31.099822: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/hacene/.cache/pypoetry/virtualenvs/ocr-id-cards-tiqq-sx6-py3.8/lib/python3.8/site-packages/cv2/../../lib64:
2022-01-05 00:28:31.099846: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-05 00:28:31.099864: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (hacene-XPS-15-9500): /proc/driver/nvidia/version does not exist
2022-01-05 00:28:31.100077: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the 

In [9]:
loaded = tf.saved_model.load("../models/model")
print(list(loaded.signatures.keys()))  # ["serving_default"]
infer = loaded.signatures["serving_default"]
print(infer.structured_outputs)

['serving_default']
{'sequential_4': TensorSpec(shape=(None, 37), dtype=tf.float32, name='sequential_4')}


In [10]:
def predict_fn(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_image(img, channels=1, expand_animations=False)
    x = tf.expand_dims(img, axis=0)
    pre = model(tf.cast(x, dtype=tf.float32))
    c = return_class(pre, CLASSES)
    return c

In [11]:
def predict_fn(image):
    img = tf.cast(image, dtype=tf.float32)
    x = tf.expand_dims(img, axis=0)
    x = tf.expand_dims(x, axis=-1)
    pre = model(x)
    c = return_class(pre, CLASSES)
    return c

In [12]:
def return_data(characters):
    model_input = [tf.cast(image, dtype=tf.float32) for image in characters]
    model_input = [tf.expand_dims(image, axis=-1) for image in model_input]
    model_input = tf.stack(model_input, axis=0)
    # predictions = model.predict(model_input)
    predictions = infer(model_input)["sequential_4"].numpy()
    outputs = return_class(predictions, CLASSES)
    line0 = outputs[0:9]
    line1 = outputs[9:24]
    line2 = outputs[24:]
    print(line1)

    id_number = "".join(line0)
    birthdate = "".join(line1[0:6])
    birthdate = birthdate[0:2] + "-" + birthdate[2:4] + "-" + birthdate[4:]
    sex = line1[7]

    exp_date = "".join(line1[8:14])
    exp_date = exp_date[0:2] + "-" + exp_date[2:4] + "-" + exp_date[4:]

    last_name = ""
    first_name = ""
    i = 0
    prec = False
    actual = False
    sep = False
    while (i < len(line2)) and not (sep):
        c = line2[i]
        prec = actual
        if c != "<":
            last_name += c
            actual = False
        else:
            last_name += " "
            actual = True
        sep = actual and prec
        i += 1

    prec = False
    actual = False
    sep = False
    while (i < len(line2)) and not (sep):
        c = line2[i]
        prec = actual
        if c != "<":
            first_name += c
            actual = False
        else:
            first_name += " "
            actual = True
        sep = actual and prec
        i += 1
    informations = {
        "first_name": first_name,
        "last_name": last_name,
        "birthdate": birthdate,
        "sex": sex,
        "id": id_number,
        "exp_date": exp_date,
    }
    return informations, predictions

In [13]:
im = cv2.imread("../data_sample/raw/33.png")
path = "./images/"

total = get_characters_from_image(im)
characters = get_used_characters(total)
informations, predictions = return_data(characters)

print("Family Name : {} ".format(informations["last_name"]))
print("Name : {} ".format(informations["first_name"]))
print("ID : {} ".format(informations["id"]))
print("Birthdate : {} ".format(informations["birthdate"]))
print("Sex: {} ".format(informations["sex"]))
print("Day of expiration : {} ".format(informations["exp_date"]))

['9' '0' '0' '7' '1' '3' '8' 'F' '3' '0' '0' '2' '1' '1' '9']
Family Name : MARTIN   
Name : MAELYS GAELLE MARIE   
ID : 348182704 
Birthdate : 90-07-13 
Sex: F 
Day of expiration : 30-02-11 
